# Description
- This notebook shows how to use the Kibot API

## Import

In [ ]:
%load_ext autoreload
%autoreload 2
import logging

import pandas as pd
import seaborn as sns

import core.config as cfg
import core.explore as exp
import core.finance as fin
import helpers.dbg as dbg
import helpers.env as env
import helpers.printing as pri
import vendors2.kibot.utils as kut

In [ ]:
print(env.get_system_signature())

pri.config_notebook()

# dbg.init_logger(verbosity=logging.DEBUG)
dbg.init_logger(verbosity=logging.INFO)
# dbg.test_logger()

_LOG = logging.getLogger(__name__)

# Metadata

## Read metadata

In [ ]:
kmd = kut.KibotMetadata()
metadata = kmd.get_metadata()
metadata

In [ ]:
metadata.sort_index()

## Read misc metadata

In [ ]:
df1 = kut.read_1min_contract_metadata()
df1.head(3)

In [ ]:
df2 = kut.read_daily_contract_metadata()
df2.head(3)

In [ ]:
df3 = kut.read_tickbidask_contract_metadata()
df3.head(3)

In [ ]:
df4 = kut.read_continuous_contract_metadata()
print(df4.head(3))

print(df4["Exchange"].unique())

In [ ]:
df4.dropna(how="all")

## Explore metadata

In [ ]:
mask = ["GAS" in d or "OIL" in d for d in df4["Description"].values]
print(sum(mask))
print(df4[mask].drop(["SymbolBase", "Size(MB)"], axis=1))

# Price data

## Read continuous daily prices for single futures

In [ ]:
s = "CL"
# nrows = None
nrows = 10000
df = kut.read_data("D", "continuous", s, nrows=nrows)
df.head(3)

## Read continuous 1-min prices for single futures

In [ ]:
s = "CL"
# nrows = None
nrows = 10000
df = kut.read_data("T", "continuous", s, nrows=nrows)
df.head(3)

In [ ]:
## Read continuous 1-min prices for multiple futures

## Read continuous daily prices for multiple futures

In [ ]:
symbols = tuple("CL NG RB BZ".split())
nrows = 10000

daily_price_dict_df = kut.read_data("D", "continuous", symbols, nrows=nrows)

daily_price_dict_df["CL"].head(3)

## Read continuous 1-min prices for multiple futures

In [ ]:
symbols = tuple("CL NG RB BZ".split())
nrows = 10000

daily_price_dict_df = kut.read_data("D", "continuous", symbols, nrows=nrows)

daily_price_dict_df["CL"].head(3)

## Read data through config API

In [ ]:
config = cfg.Config.from_env()

if config is None:
    config = cfg.Config()
    config_tmp = config.add_subconfig("read_data")
    # Use the data from S3.
    file_name = hs3.get_path() + "/kibot/All_Futures_Contracts_1min/ES.csv.gz"
    config_tmp["file_name"] = file_name
    config_tmp["nrows"] = 100000

_LOG.info(config)

In [ ]:
def read_data_from_config(config):
    _LOG.info("Reading data ...")
    config.check_params(["file_name"])
    return kut._read_data(config["file_name"], config.get("nrows", None))


df = read_data_from_config(config["read_data"])

_LOG.info("df.shape=%s", df.shape)
_LOG.info("datetimes=[%s, %s]", df.index[0], df.index[-1])
_LOG.info("df=\n%s", df.head(3))

## Read raw data directly from S3

In [ ]:
s = "CL"
file_name = (
    hs3.get_path() + "/kibot/All_Futures_Continuous_Contracts_1min/%s.csv.gz" % s
)
nrows = 10000

df = pd.read_csv(file_name, header=None, parse_dates=[0], nrows=nrows)
# df.columns = "datetime open high low close vol".split()
df.head(3)

# Return computation

## 1-min for single futures

In [ ]:
# TODO(gp)

## 1-min for multiple futures

In [ ]:
# Read multiple futures.
symbols = tuple("CL NG RB BZ".split())
nrows = 100000
min_price_dict_df = kut.read_data(
    "T", "continuous", symbols, ext="csv", nrows=nrows
)
_LOG.info("keys=%s", min_price_dict_df.keys())
min_price_dict_df["CL"].tail(3)

### Compute returns ret_0

In [ ]:
def compute_ret_0_from_multiple_1min_prices(price_dict_df, mode):
    dbg.dassert_isinstance(price_dict_df, dict)
    rets = []
    for s, price_df in price_dict_df.items():
        _LOG.debug("Processing s=%s", s)
        rets_tmp = fin.compute_ret_0(price_df["open"], mode)
        rets_tmp = pd.DataFrame(rets_tmp)
        rets_tmp.columns = ["%s_ret_0" % s]
        rets.append(rets_tmp)
    rets = pd.concat(rets, sort=True, axis=1)
    return rets


mode = "pct_change"
min_rets = compute_ret_0_from_multiple_1min_prices(min_price_dict_df, mode)



min_rets.head(3)

In [ ]:
min_rets.fillna(0.0).resample("1D").sum().cumsum().plot()

### Resample to 1min

In [ ]:
# Resample to 1min.
_LOG.info("## Before resampling")
exp.report_zero_nan_inf_stats(min_rets)

In [ ]:
exp.plot_non_na_cols(min_rets.resample("1D").sum())

In [ ]:
min_rets = fin.resample_1min(min_rets, skip_weekends=False)

_LOG.info("## After resampling")
exp.report_zero_nan_inf_stats(min_rets)

min_rets.fillna(0.0, inplace=True)

### z-scoring

In [ ]:
zscore_com = 28
min_zrets = fin.zscore(
    min_rets, com=zscore_com, demean=False, standardize=True, delay=1
)
min_zrets.columns = [c.replace("ret_", "zret_") for c in min_zrets.columns]
min_zrets.dropna().head(3)

In [ ]:
min_zrets.fillna(0.0).resample("1D").sum().cumsum().plot()

In [ ]:
annot = True
stocks_corr = min_rets.dropna().corr()

sns.clustermap(stocks_corr, annot=annot)

## Daily for single futures

## Daily for multiple futures